# 很抱歉，這個notebook我也忘記用在哪了 😢

# Aggregate average monthly transaction amounts for each merchant

## Pyspark set up

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [11]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

## Read in data
### Data relating to each transaction

In [12]:
transaction_20210828_20220227_sdf = spark.read.parquet("../data/curated/transactions_20210828_20220227")

### Giving each records the year and month it is happening

In [13]:
data = transaction_20210828_20220227_sdf.withColumn("Year", F.year("order_datetime"))\
        .withColumn("Month", F.month("order_datetime"))

## Aggregation
### Group transaction records of each merchant by its year and month
#### Find total sale amounts, number of transactions and number of customers
1\. total sale amounts == sum_transactions \
2\. number of transactions == number_of_customers \
3\. number of customers == distinct_customers

In [14]:
transact_agg_by_month = data.groupBy("Year", "Month", "merchant_abn")\
        .agg(F.sum("dollar_value")\
        .alias("sum_transactions"), \
         F.count("order_id").alias("number_of_customers"), \
         F.countDistinct("user_id").alias("distinct_customers"))

In [15]:
transact_agg_by_month

Year,Month,merchant_abn,sum_transactions,number_of_customers,distinct_customers
2021,9,20569508100,10099.236167068768,131,131
2021,12,19933438190,65121.04160676313,4671,4245
2021,10,46970913786,39502.63705626766,202,197
2021,10,71247553576,79348.37054465171,238,236
2021,12,96965554802,13385.328780379374,110,110
2021,11,67915848101,56376.517798328365,156,156
2021,11,90543168331,636310.2107110615,2444,2337
2022,1,22033359776,356058.72007584944,926,913
2022,1,99976658299,134429.57385066163,882,868
2021,9,72553304202,21054.74136332652,329,328


### Group overall transactions for a merchant 

In [33]:
transact_agg_by_monthly_average = transact_agg_by_month.groupby("merchant_abn")\
    .agg(F.count("Month").alias("Number_of_Month"),\
    F.sum("sum_transactions").alias("total_trans"))

### Find the average monthly transaction money 

In [34]:
final_df = transact_agg_by_monthly_average.withColumn('average_monthly_transaction', transact_agg_by_monthly_average.total_trans/transact_agg_by_monthly_average.Number_of_Month)